### Extract the features of the last hidden layer

In [4]:
from torchvision.models import resnet50
import torch
import numpy as np


In [9]:
# load the pre-trained model
model = resnet50(pretrained=True)

# evaluate the model
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
# remove the last layer (the classification layer, fc)

model = torch.nn.Sequential(*(list(model.children())[:-1]))
model.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [13]:
# load the image
from PIL import Image
from torchvision import transforms

img = Image.open('../data/images.jpeg')

In [16]:
# apply the same transformations as the ones used during training

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [30]:
image_tensor = transform(img).unsqueeze(0) # add 1 more dimension to simulate a batch

image_tensor.shape # the first dimension is the batch size

torch.Size([1, 3, 224, 224])

In [22]:
# gradients are not calculated as we are using a pre-trained model
# we only want the features from the image based on the pre-trained model
with torch.no_grad():
    features = model(image_tensor)

print(features.shape)

torch.Size([1, 2048, 1, 1])


In [27]:
features = features.squeeze(0).numpy() 

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [29]:
print("Feature vector shape: ", features.shape)

Feature vector shape:  (2048, 1, 1)


### Generalize for multiple images

In [ ]:
import os

def extract_features(image_path, model, transform):
    img = Image.open(image_path)
    image_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        features = model(image_tensor)
    return features.squeeze(0).numpy()

# training images
image_dir = '../data/training_images/'

all_features = []

# for each image
for image_name in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_name)
    features = extract_features(image_path, model, transform)
    all_features.append(features)

# Convert list to numpy array
all_features = np.array(all_features)
print("All features shape: ", all_features.shape)